# Welcome to the Lab 🥼🧪

## Investor Share of Resale Listings

In this notebook, we will go over how much of an impact investors have on the resale listing market. 

We will create a chart like this: 

![Chart](assets/atlanta_investor_share.png)

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/investor_share_of_resale_listings.ipynb)

In [ ]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# initialize client
client = ParclLabsClient(api_key=api_key)

In [ ]:
# retrieving info for the US Parcl
us = client.search_markets.retrieve(
    query='United States',
    as_dataframe=True
)
us.head()

In [ ]:
# retrieving info for the Tampa Parcl
tampa = client.search_markets.retrieve(
    query='Tampa',
    location_type='CBSA',
    as_dataframe=True
)
tampa.head()

In [ ]:
# retrieving info for the Phoenix Parcl
phoenix = client.search_markets.retrieve(
    query='Phoenix',
    location_type='CBSA',
    as_dataframe=True
)

phoenix.head()

In [ ]:
# retrieving info for the Atlanta Parcl
atlanta = client.search_markets.retrieve(
    query='Atlanta',
    location_type='CBSA',
    as_dataframe=True
)

atlanta.head()

In [ ]:
# retrieving info for the San Francisco Parcl
sf = client.search_markets.retrieve(
    query='San Francisco',
    location_type='CBSA',
    as_dataframe=True
)

sf.head()

In [ ]:
# retrieving info for the Miami Parcl
miami = client.search_markets.retrieve(
    query='Miami',
    location_type='CBSA',
    as_dataframe=True,
)

miami = miami.loc[miami['name'].str.contains('Fl')]
miami

In [ ]:
# retrieving info for the Austin Parcl
austin = client.search_markets.retrieve(
    query='Austin',
    location_type='CBSA',
    as_dataframe=True
)

austin = austin.loc[austin['name'].str.contains('Tx')]
austin.head()

In [ ]:
# retrieving info for the Vegas Parcl
vegas = client.search_markets.retrieve(
    query='Las Vegas',
    location_type='CBSA',
    as_dataframe=True
)

vegas = vegas.loc[vegas['name'].str.contains('Nv')]
vegas.head()

In [ ]:
# retrieving info for the East North Central census region Parcls
east_north_central_cities = client.search_markets.retrieve(
    location_type='CITY',
    region='EAST_NORTH_CENTRAL',
    params={
        'limit': 1000
    },
    as_dataframe=True
)

east_north_central_cities.head()

In [ ]:
# concatenate all Parcls of interest
locations = pd.concat([us, phoenix, sf, atlanta, vegas])
locations

In [ ]:
location_ids = locations['parcl_id'].tolist()
location_ids

In [ ]:
# now lets get the investor share of listings for multiple markets

investor_listings = client.investor_metrics_new_listings_for_sale_rolling_counts.retrieve_many(
    parcl_ids=location_ids,
    params={
        'limit': 300
    },
    as_dataframe=True
)

investor_listings['pct_for_sale_market'] = investor_listings['pct_for_sale_market']/100
investor_listings.head(4)

In [ ]:
# lets focus on 30 day period
investor_listings_30 = investor_listings.loc[investor_listings['period'] == 'rolling_60_day']
investor_listings_30

In [ ]:
# join against location name
locations.name.unique()

# cleanup names
name_map = {
    'United States Of America': 'USA', 
    'Tampa-St. Petersburg-Clearwater, Fl': 'Tampa',
    'Atlanta-Sandy Springs-Alpharetta, Ga': "Atlanta",
    'San Francisco-Oakland-Berkeley, Ca': "San Francisco",
    'Miami-Fort Lauderdale-Pompano Beach, Fl': "Miami",
    'Austin-Round Rock-Georgetown, Tx': "Austin",
    "Atlanta-Sandy Springs-Alpharetta, Ga": "Atlanta", 
    "Las Vegas-Henderson-Paradise, Nv": "Vegas"
}

locations['name'] = locations['name'].replace(name_map)

investor_listings_30 = pd.merge(investor_listings_30, locations[['parcl_id', 'name']], on='parcl_id')
investor_listings_30

In [ ]:
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white.png'
}

# set charting constants
labs_logo_dict = dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0.5,  # Centering the logo below the title
        y=1.02,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="center",
        yanchor="bottom"
)

In [ ]:
# create chart, overlay the logo and format our viz
fig = px.line(
    investor_listings_30,
    x='date',
    y='pct_for_sale_market',
    color='name',  # This creates separate lines for each period
    title=f'Investor Share of New Resale Listings Activity (Rolling 60 Days)',
    labels={'pct_for_sale_market': '% of New Listings by Investors'},
    line_shape='linear'  # 'spline' for smooth lines, if preferred
)

fig.add_layout_image(labs_logo_dict)

fig.update_layout(
    margin=dict(b=100),
    height=900,
    width=1600,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Market'
    ),
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    xaxis_title='date' 
)

fig.update_yaxes(tickformat=".0%")

color_map = {
    'rolling_30_day': '#448CF2',
    'rolling_90_day': '#FFFFFF',
}

fig.update_traces(
    line=dict(width=3),
    mode='lines+markers',
    marker=dict(size=5)
)


for trace in fig.data:
    if trace.name in color_map:
        trace.line.color = color_map[trace.name]
        trace.marker.color = color_map[trace.name]

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Show the plot
fig.show() # Vegas, Atlanta, San Fran, USA